In [18]:
# We will include captions all along the way
# Import required modules
import pandas as pd
import matplotlib as plt

#Obtain datasets from file
fhfa = pd.read_csv("Databases/fhfa_hpi.csv")
laus = pd.read_csv("Databases/laus.csv")
real_estate = pd.read_csv("Databases/kaggle_real_estate.csv")

### Cleaning Data ###


In [126]:
#Intial Processing of HPI
df_hpi = fhfa.copy()

#Rename Columns
df_hpi.rename(columns={"place_name":"region",
                       "yr":"year",
                       "period" : "month"},
                       inplace=True)

#Inspect the null values
display(df_hpi.isnull().sum())
display(df_hpi[df_hpi.index_sa.isnull()])

#Handle those null values using a forward fill from nsa to sa
df_hpi = df_hpi.fillna(method='ffill', axis=1)
display(df_hpi.isnull().sum())
display(df_hpi.iloc[4170:, :])

#Create a date column using month and year and then drop month and year
df_hpi["date"] = pd.to_datetime(df_hpi['year']*100 + df_hpi['month'], format='%Y%m').dt.to_period("M")
df_hpi.drop(columns=["year", "month"], inplace=True)
df_hpi.drop_duplicates(keep='first', inplace=True)

display(df_hpi)

hpi_type          0
hpi_flavor        0
frequency         0
level             0
region            0
place_id          0
year              0
month             0
index_nsa         0
index_sa      88320
dtype: int64

,hpi_type,hpi_flavor,frequency,level,region,place_id,year,month,index_nsa,index_sa
4170,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1986,2,108.06,NaN
4171,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1986,3,108.28,NaN
4172,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1986,4,94.80,NaN
4173,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1987,1,101.63,NaN
4174,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1987,2,100.89,NaN
...,...,...,...,...,...,...,...,...,...,...
132239,developmental,all-transactions,quarterly,Puerto Rico,Puerto Rico,PR,2024,3,249.92,NaN
132240,developmental,all-transactions,quarterly,Puerto Rico,Puerto Rico,PR,2024,4,250.97,NaN
132241,developmental,all-transactions,quarterly,Puerto Rico,Puerto Rico,PR,2025,1,247.17,NaN
132242,developmental,all-transactions,quarterly,Puerto Rico,Puerto Rico,PR,2025,2,252.11,NaN


C:\Users\zimin\AppData\Local\Temp\ipykernel_61032\400998330.py:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_hpi = df_hpi.fillna(method='ffill', axis=1)


hpi_type      0
hpi_flavor    0
frequency     0
level         0
region        0
place_id      0
year          0
month         0
index_nsa     0
index_sa      0
dtype: int64

,hpi_type,hpi_flavor,frequency,level,region,place_id,year,month,index_nsa,index_sa
4170,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1986,2,108.06,108.06
4171,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1986,3,108.28,108.28
4172,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1986,4,94.8,94.8
4173,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1987,1,101.63,101.63
4174,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1987,2,100.89,100.89
...,...,...,...,...,...,...,...,...,...,...
132358,developmental,purchase-only,quarterly,Puerto Rico,Puerto Rico,PR,2024,3,236.93,229.3
132359,developmental,purchase-only,quarterly,Puerto Rico,Puerto Rico,PR,2024,4,230.09,234.64
132360,developmental,purchase-only,quarterly,Puerto Rico,Puerto Rico,PR,2025,1,232.42,237.66
132361,developmental,purchase-only,quarterly,Puerto Rico,Puerto Rico,PR,2025,2,252.34,250.76


,hpi_type,hpi_flavor,frequency,level,region,place_id,index_nsa,index_sa,date
0,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,100.0,100.0,1991-01
1,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,100.87,100.87,1991-02
2,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,101.32,100.9,1991-03
3,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,101.73,100.96,1991-04
4,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,102.32,101.31,1991-05
...,...,...,...,...,...,...,...,...,...
132358,developmental,purchase-only,quarterly,Puerto Rico,Puerto Rico,PR,236.93,229.3,2024-03
132359,developmental,purchase-only,quarterly,Puerto Rico,Puerto Rico,PR,230.09,234.64,2024-04
132360,developmental,purchase-only,quarterly,Puerto Rico,Puerto Rico,PR,232.42,237.66,2025-01
132361,developmental,purchase-only,quarterly,Puerto Rico,Puerto Rico,PR,252.34,250.76,2025-02


In [ ]:
#Initial Processing of LAUS
df_laus = laus.copy()
df_laus.head(5)

df_laus.rename(columns={"Area Name": "region",
                        "Area Type": "area_type",
                        "Year": "year",
                        "Month": "month",
                        "Date_Numeric": "date",
                        "Seasonally Adjusted(Y/N)":"seasonally_adjusted",
                        "Status": "status",
                        "Labor Force": "labor_force",
                        "Employment": "employed",
                        "Unemployment": "unemployed",
                        "Unemployment Rate" : "unemployment_rate",
                        "Benchmark" : "benchmark"},
                        inplace=True)
df_laus['date'] = pd.to_datetime(df_laus['date'], format='%m/%Y').dt.to_period("M")
df_laus.drop(columns=["year","month"], inplace=True)
df_laus.drop_duplicates(keep='first',inplace=True)
display(df_laus)

,region,area_type,date,seasonally_adjusted,status,labor_force,employed,unemployed,unemployment_rate,benchmark
0,California,State,1976-01,N,Final,9672362,8668016,1004346,10.4,2023
1,California,State,1976-01,Y,Final,9774280,8875685,898595,9.2,2023
2,California,State,1976-02,N,Final,9684440,8704564,979876,10.1,2023
3,California,State,1976-02,Y,Final,9768885,8871553,897332,9.2,2023
4,California,State,1976-03,N,Final,9689626,8776344,913282,9.4,2023
...,...,...,...,...,...,...,...,...,...,...
215043,Yolo County,County,2025-08,N,Preliminary,113800,107100,6600,5.8,2024
215044,Yuba City MSA,Metropolitan Area,2025-07,N,Final,83700,76900,6800,8.1,2024
215045,Yuba City MSA,Metropolitan Area,2025-08,N,Preliminary,84600,78400,6200,7.3,2024
215046,Yuba County,County,2025-07,N,Final,36200,33500,2700,7.4,2024


In [127]:
#Housing value initial processing
df_real_estate = real_estate.copy()

#Rename the columns for easier processing
df_real_estate.rename(columns={"RegionID": "region_id",
                               "SizeRank" : "size_rank",
                               "RegionName" : "region",
                               "StateName" : "state"},
                               inplace=True)

#undo the pivot on the home value
df_real_estate_home_value = df_real_estate.copy()
df_real_estate_home_value.rename(columns={"2018-02-HomeValue" : "2018-02",
                               "2018-03-HomeValue" : "2018-03",
                               "2024-03-HomeValue" : "2024-03",
                               "2024-04-HomeValue" : "2024-04",
                               "2024-05-HomeValue" : "2024-05"},
                               inplace=True)
df_real_estate_home_value = pd.melt(df_real_estate_home_value, id_vars=['region_id', 'size_rank', 'region', 'state'], value_vars=['2018-02','2018-03','2024-03','2024-04','2024-05'], var_name='date', value_name='home_value')
display(df_real_estate_home_value)

#undo the pivot on the DaysPending value
df_real_estate_days_pending = df_real_estate.copy()
df_real_estate_days_pending.rename(columns={"2018-02-DaysPending" : "2018-02",
                               "2018-03-DaysPending" : "2018-03",
                               "2024-03-DaysPending" : "2024-03",
                               "2024-04-DaysPending" : "2024-04",
                               "2024-05-DaysPending" : "2024-05"},
                               inplace=True)
df_real_estate_days_pending = pd.melt(df_real_estate_days_pending, id_vars=['region_id', 'size_rank', 'region', 'state'], value_vars=['2018-02','2018-03','2024-03','2024-04','2024-05'], var_name='date', value_name='days_pending')
display(df_real_estate_days_pending)

#undo the pivot on the cut_raw value
df_real_estate_cut_raw = df_real_estate.copy()
df_real_estate_cut_raw.rename(columns={"2018-02-CutRaw" : "2018-02",
                               "2018-03-CutRaw" : "2018-03",
                               "2024-03-CutRaw" : "2024-03",
                               "2024-04-CutRaw" : "2024-04",
                               "2024-05-CutRaw" : "2024-05"},
                               inplace=True)
df_real_estate_cut_raw = pd.melt(df_real_estate_cut_raw, id_vars=['region_id', 'size_rank', 'region', 'state'], value_vars=['2018-02','2018-03','2024-03','2024-04','2024-05'], var_name='date', value_name='cut_raw')
display(df_real_estate_cut_raw)

df_real_estate = pd.merge(df_real_estate_home_value, df_real_estate_days_pending, how='inner', on=['date','region_id','size_rank','region','state'])
df_real_estate = pd.merge(df_real_estate, df_real_estate_cut_raw, how='left', on=['date','region_id','size_rank','region','state'])
#df_real_estate.dropna(inplace=True)
df_real_estate['date'] = pd.to_datetime(df_real_estate['date'], format='%Y-%m').dt.to_period("M")
#df_real_estate['days_pending'] = df_real_estate['days_pending'].astype(int)
#df_real_estate['cut_raw'] = df_real_estate['cut_raw'].astype(int)
df_real_estate['home_value'] = df_real_estate['home_value'].round(2)
df_real_estate.drop_duplicates(keep='first', inplace=True)
df_real_estate.state.fillna("Unknown", inplace=True)
df_real_estate[df_real_estate['state'] == 'Unknown']


,region_id,size_rank,region,state,date,home_value
0,102001,0,United States,NaN,2018-02,225746.614318
1,394913,1,"New York, NY",NY,2018-02,468313.430126
2,753899,2,"Los Angeles, CA",CA,2018-02,629019.165765
3,394463,3,"Chicago, IL",IL,2018-02,229741.266973
4,394514,4,"Dallas, TX",TX,2018-02,241533.231800
...,...,...,...,...,...,...
4470,753929,935,"Zapata, TX",TX,2024-05,119697.355634
4471,394743,936,"Ketchikan, AK",AK,2024-05,389962.681122
4472,753874,937,"Craig, CO",CO,2024-05,287028.957463
4473,395188,938,"Vernon, TX",TX,2024-05,92149.470030


,region_id,size_rank,region,state,date,days_pending
0,102001,0,United States,NaN,2018-02,52.0
1,394913,1,"New York, NY",NY,2018-02,68.0
2,753899,2,"Los Angeles, CA",CA,2018-02,33.0
3,394463,3,"Chicago, IL",IL,2018-02,52.0
4,394514,4,"Dallas, TX",TX,2018-02,46.0
...,...,...,...,...,...,...
4470,753929,935,"Zapata, TX",TX,2024-05,NaN
4471,394743,936,"Ketchikan, AK",AK,2024-05,NaN
4472,753874,937,"Craig, CO",CO,2024-05,NaN
4473,395188,938,"Vernon, TX",TX,2024-05,NaN


,region_id,size_rank,region,state,date,cut_raw
0,102001,0,United States,NaN,2018-02,7600.0
1,394913,1,"New York, NY",NY,2018-02,16000.0
2,753899,2,"Los Angeles, CA",CA,2018-02,20000.0
3,394463,3,"Chicago, IL",IL,2018-02,8000.0
4,394514,4,"Dallas, TX",TX,2018-02,8562.0
...,...,...,...,...,...,...
4470,753929,935,"Zapata, TX",TX,2024-05,NaN
4471,394743,936,"Ketchikan, AK",AK,2024-05,NaN
4472,753874,937,"Craig, CO",CO,2024-05,NaN
4473,395188,938,"Vernon, TX",TX,2024-05,NaN


,region_id,size_rank,region,state,date,home_value,days_pending,cut_raw
0,102001,0,United States,Unknown,2018-02,225746.61,52.0,7600.0
895,102001,0,United States,Unknown,2018-03,227155.65,48.0,8900.0
1790,102001,0,United States,Unknown,2024-03,357374.36,42.0,10000.0
2685,102001,0,United States,Unknown,2024-04,359240.11,38.0,10000.0
3580,102001,0,United States,Unknown,2024-05,360681.29,37.0,10000.0


### Filtering Subsets